In [1]:
import pandas
import numpy as np
import re 
from pyvi.ViTokenizer import ViTokenizer
import tensorflow as tf
import pickle
from keras.preprocessing import sequence
import requests


In [2]:
STOPWORDS = 'stop_word.txt'

with open(STOPWORDS, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip("\n")
        stopwords.append(dd)
    stopwords = set(stopwords)
#Filter stop words
def filter_stop_words(train_sentences, stopwords):
    new_sent = [word for word in train_sentences.split() if word not in stopwords]
    train_sentences = ' '.join(new_sent)
    return train_sentences

#Delete Emoji
def deEmojify(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

#Preprocess
def preprocess(text, tokenized=True, lowercase=True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercase else text
    return text

#-------Extract Features------------
def pre_process_features(X, tokenized=True, lowercase=True):
    X = [preprocess(str(X), tokenized=tokenized, lowercase=lowercase)]
    for idx, ele in enumerate(X):
        if not ele:
            np.delete(X, idx)
    return X

In [3]:
test = "mày ngu quá"
preprocess_test = pre_process_features(test,tokenized=True,lowercase=True)
print(preprocess_test)

['mày ngu']


In [4]:
def make_features(X, tokenizer, is_one_hot_label=True,sequence_length=100):
    X = tokenizer.texts_to_sequences(X)
    X = sequence.pad_sequences(X, maxlen=sequence_length)
    return X

In [5]:
model = tf.keras.models.load_model('Text_CNN_model_v13.h5')
file = open("tokenizer.pickle","rb")
tokenizer = pickle.load(file)

process_test = make_features(preprocess_test, tokenizer, is_one_hot_label=False)

#predict = model.predict(process_test, batch_size = 0, verbose =0)
#print(predict.argmax(axis = -1))

In [8]:
import json
headers = {"content-type": "application/json"}
data = json.dumps({"instances":process_test.tolist()})

json_response = requests.post("http://localhost:8605/v1/models/twitter_model:predict",data=data,headers=headers)

In [12]:
predictions = json.loads(json_response.text)['predictions']
print(np.argmax(predictions,axis=-1))

[2]
